In [46]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont  # 載入 PIL 相關函式庫

def setting(img,h,w):
    img = cv2.resize(img, (w//2, h//2), interpolation=cv2.INTER_LINEAR) #important
    #縮小圖片，記住這邊的 interpolation 選擇很重要，如果不確定建議可以都試試找最棒的

    return img

def coint(img):
    thresh = 170
    maxval = 255
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #灰階化，方便閱讀
    ret, gray = cv2.threshold(gray, 190, 255, cv2.THRESH_BINARY) #二值化，90 是慢慢設定的

    gray = cv2.erode(gray, np.ones((4,4)), iterations=3) #侵蝕，裡面矩陣與 iterations 細調
    gray = cv2.dilate(gray, np.ones((2,2)), iterations=1) #膨脹，裡面矩陣與 iterations 細調
    
    ret,gray = cv2.threshold(gray, thresh, maxval, cv2.THRESH_BINARY_INV)#反轉黑白
    
    
    
    #連通物件，設八通道這樣才嚴謹

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(gray, connectivity=8)
    print(stats)  #檢查所有區塊

    ans = 0 #輸出多少錢

    for it in stats:
        itX = it[0]+it[2] #寬度
        itY = it[1]+it[3] #高度
        cv2.rectangle(img, (it[0], it[1]), (itX, itY), (200, 31, 31), 2)
        #一開始可以使用，這樣她會畫出所有的區塊來讓你判斷有沒有區塊沒畫到或連在一起

        #下面的長度是我自己去慢慢調整出來得
        if(it[2] >= 50 and it[2] < 64 and it[3] > 50): #1 dallars
            cv2.rectangle(img, (it[0], it[1]), (itX, itY), (255, 0, 255), 2) #BGR 紫
            ans += 1
            print("一元有")
            print(it)
        if(it[2] >= 64 and it[2] < 70 and it[3] > 50): #5 dallars
            cv2.rectangle(img, (it[0], it[1]), (itX, itY), (0, 165, 255), 2) #BGR 橘
            ans += 5
            print("五元有")
            print(it)
        if(it[2] >= 70 and it[2] < 76 and it[3] > 50): #10 dallars
            cv2.rectangle(img, (it[0], it[1]), (itX, itY), (0, 255, 255), 2) #BGR 黃
            ans += 10
            print("十元有")
            print(it)
        if(it[2] >= 76 and it[2] < 95 and it[3] > 50): #50 dallars  
            cv2.rectangle(img, (it[0], it[1]), (itX, itY), (0, 255, 0), 2) #BGR 綠
            ans += 50
            print("五十元有")
            print(it)
            
    text = "照片裡有 %s 元" %ans
    fontpath = 'NotoSansTC-Regular.ttf'
    imgPil = Image.fromarray(img)                # 將 img 轉換成 PIL 影像
    draw = ImageDraw.Draw(imgPil)                # 準備開始畫畫
    draw.text((50, 0), text, fill=(255, 255, 0), font=ImageFont.truetype(fontpath, 50) )  # 畫入文字
    img = np.array(imgPil)
    cv2.imshow("img", img)
    cv2.imshow('gray.jpg', gray)
    cv2.imwrite('ans.jpg', img)
    cv2.imwrite('gray.jpg', gray)
    cv2.waitKey(0)
    return ans

#cv2.imshow("gray", gray) 
#thresh = 127
#maxval = 255

%store -r name
print(name)
img = cv2.imread(name) #輸入資料
# img = cv2.imread("captured_photo_1703528403.jpg") #輸入資料

h, w, ch = img.shape #圖片高、寬、通道

img=setting(img,h,w)

ans=coint(img)

print("圖上共有 ",ans ,"元") #輸出答案 




[[     0      0    640    360 202254]
 [    67      9     12      9    108]
 [   509     10      9      9     81]
 [   170     67     60     64   3223]
 [   250     73     64     67   3632]
 [   342     75     58     64   3219]
 [   111    117     66     69   3755]
 [   289    145     73     78   4671]
 [   182    183     79     82   5400]
 [   375    192     71     76   4057]]
一元有
[ 170   67   60   64 3223]
五元有
[ 250   73   64   67 3632]
一元有
[ 342   75   58   64 3219]
五元有
[ 111  117   66   69 3755]
十元有
[ 289  145   73   78 4671]
五十元有
[ 182  183   79   82 5400]
十元有
[ 375  192   71   76 4057]
圖上共有  82 元
